In [21]:
import numpy as np
import pandas as pd

In [1]:
# 데이터 불러오기
import pandas as pd
dataset = pd.read_csv('./total_price.csv')

# Del Unnamed: 0
del dataset['Unnamed: 0']

# l1~l4 line Setting
for step in ['l1', 'l2', 'l3', 'l4'] : 
    dataset[step] = dataset[step].apply(lambda x : str(x)).apply(lambda x : x[1:]).apply(lambda x : float(x))
    
# Warning Message
import warnings
warnings.filterwarnings(action='ignore') 

# 작업하는 모든 코드 리스트
code_list = dataset.code.unique()

In [44]:
# 오늘의 예측 데이터프레임
today_df = pd.DataFrame()

# 진행중 계산
step = 1

# 모든 코드 진행
for code in code_list[:1] : 
    
    # 코드 변환
    code_1 = code
    code_2 = str(code).rjust(6, '0')
    
    # 진행 결과문 표시
    if step % 250 == 1 :
        print('현재 {:4}번째 종목 진행중...({:5.2f}%)' .format(step, 100*step/len(code_list)))
    step += 1
    
    # 코드에 해당하는 데이터만 추출
    data = dataset[dataset['code'] == code_1]
    
    # 데이터 정규화하기
    mean = data[:700].mean(axis=0)
    data -= mean
    std = data[:700].std(axis=0)
    data /= std
    
    # code 다시 되돌리기
    data['code'] = code
    
    # 제네레이터 함수
    def generator(data, lookback, delay, min_index, max_index,
                  shuffle=False, batch_size=128, step=6):
        if max_index is None:
            max_index = len(data) - delay - 1
        i = min_index + lookback
        while 1:
            if shuffle:
                rows = np.random.randint(
                    min_index + lookback, max_index, size=batch_size)
            else:
                if i + batch_size >= max_index:
                    i = min_index + lookback
                rows = np.arange(i, min(i + batch_size, max_index))
                i += len(rows)

            samples = np.zeros((len(rows),
                               lookback // step,
                               data.shape[-1]))
            targets = np.zeros((len(rows),))
            for j, row in enumerate(rows):
                indices = range(rows[j] - lookback, rows[j], step)
                samples[j] = data[indices]
                targets[j] = data[rows[j] + delay][1]
            yield samples, targets
         
    # 제네레이터 하이퍼파라미터 생성
    lookback = 5
    step = 1
    delay = 1
    batch_size = 64
    
    # 제네레이터 적용
    lookback = 5
    step = 1
    delay = 1
    batch_size = 64

    train_gen = generator(data,
                          lookback=lookback,
                          delay=delay,
                          min_index=0,
                          max_index=700,
                          shuffle=True,
                          step=step, 
                          batch_size=batch_size)
    val_gen = generator(data,
                        lookback=lookback,
                        delay=delay,
                        min_index=701,
                        max_index=800,
                        step=step,
                        batch_size=batch_size)
    test_gen = generator(data,
                         lookback=lookback,
                         delay=delay,
                         min_index=801,
                         max_index=None,
                         step=step,
                         batch_size=batch_size)

    # 전체 검증 세트를 순회하기 위해 val_gen에서 추출할 횟수
    val_steps = (800 - 701 - lookback) // batch_size

    # 전체 테스트 세트를 순회하기 위해 test_gen에서 추출할 횟수
    test_steps = (len(data) - 801 - lookback) // batch_size
    
    # Model
    from keras.models import Sequential
    from keras import layers
    from keras.optimizers import RMSprop

    model = Sequential()
    model.add(layers.GRU(32,
                         dropout=0.1,
                         recurrent_dropout=0.5,
                         return_sequences=True,
                         input_shape=(None, data.shape[-1])))
    model.add(layers.GRU(64, activation='relu',
                         dropout=0.1, 
                         recurrent_dropout=0.5))
    model.add(layers.Dense(1))

    model.compile(optimizer=RMSprop(), loss='mae')
    history = model.fit_generator(train_gen,
                                  steps_per_epoch=500,
                                  epochs=40,
                                  validation_data=val_gen,
                                  validation_steps=val_steps)

현재    1번째 종목 진행중...( 0.05%)
Epoch 1/40


KeyError: "None of [Int64Index([234, 235, 236, 237, 238], dtype='int64')] are in the [columns]"